### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv.    
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок.    
*(Под временем жизни понимается разница между максимальным и минимальным значением столбца    
timestamp для данного значения userId.)*

In [1]:
import pandas as pd

In [2]:
pd.read_csv('ml-latest-small_2/ratings.csv').head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
# посмотреть
ratings = pd.read_csv('ml-latest-small_2/ratings.csv')
ratings.loc[ratings['userId'] == 567].count()

userId       34
movieId      34
rating       34
timestamp    34
dtype: int64

In [4]:
ratings['userId'].value_counts().head()

547    2391
564    1868
624    1735
15     1700
73     1610
Name: userId, dtype: int64

In [39]:
# выбираем > 100

users = ratings.groupby('userId').count().reset_index()
active_users = users[users['rating'] > 100]
active_users = active_users[['userId','rating']]
active_users.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [50]:
#считаем время жизни пользователей

timestamps = ratings.groupby('userId').agg([min, max]).reset_index()
timestamps['lifetime'] = timestamps['timestamp']['max'] - timestamps['timestamp']['min']
timestamps = timestamps[['userId','lifetime']]
timestamps.head()

,userId,lifetime
,,
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [51]:
# объединяем (предупреждение в конкретном случае не влияет на результат)

active_users_mean = active_users.merge(timestamps, on = 'userId', how = 'left')
active_users_mean.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,userId,rating,"(lifetime, )"
0,4,204,203560
1,8,116,85187
2,15,1700,471393496
3,17,363,8053
4,19,423,5282
5,21,162,1365432
6,22,220,91491
7,23,726,18342129
8,26,172,20267261
9,30,1011,115852276


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).       
Необходимо сформировать две таблицы:   
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента   
- аналогичную таблицу по типам выручки с указанием адреса клиента   
*(Обратите внимание, что в процессе объединения таблиц данные не должны теряться.)*

In [52]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [53]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [54]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [55]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [56]:
# вариант 1

client_id = client_base[['client_id']]
client_id

,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


In [65]:
(client_id
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [66]:
revenue = (client_id
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)
revenue.fillna(0) 

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [85]:
# Вариант 2 (без адреса)

revenue = (client_base
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)
revenue = revenue[['client_id','rzd_revenue','auto_revenue','air_revenue']]
revenue.fillna(0) 


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [72]:
# С адресом
# Вариант 2

revenue = (client_base
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)
revenue.fillna(0) 

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [93]:
# считаем общую выручку 1

revenue = (client_base
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)

revenue['total_revenue'] = revenue.fillna(0)['rzd_revenue'] + revenue.fillna(0)['auto_revenue'] + revenue.fillna(0)['air_revenue']
revenue.fillna(0)

# все по столбцам: revenue.append(revenue.sum(axis=0), ignore_index=True)

,client_id,address,rzd_revenue,auto_revenue,air_revenue,total_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0,1093.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0,2810.0
2,113,Левобережная 1а,10283.0,57483.0,0.0,67766.0
3,114,Мира 14,5774.0,83.0,0.0,5857.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0,1974.0
5,116,Строителей 18,0.0,4834.0,4.0,4838.0
6,117,Панфиловская 33,0.0,98.0,13.0,111.0
7,118,Мастеркова 4,0.0,0.0,173.0,173.0


In [99]:
# считаем общую выручку 2 (игнорируем суммирование client_id в нижней строке)

revenue = (client_base
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
)
revenue = revenue[['client_id','rzd_revenue','auto_revenue','air_revenue']]
revenue['total_revenue'] = revenue.fillna(0)['rzd_revenue'] + revenue.fillna(0)['auto_revenue'] + revenue.fillna(0)['air_revenue']
total = revenue.append(revenue.sum(axis=0), ignore_index=True)
total.fillna(0)



,client_id,rzd_revenue,auto_revenue,air_revenue,total_revenue
0,111.0,1093.0,0.0,0.0,1093.0
1,112.0,2810.0,0.0,0.0,2810.0
2,113.0,10283.0,57483.0,0.0,67766.0
3,114.0,5774.0,83.0,0.0,5857.0
4,115.0,981.0,912.0,81.0,1974.0
5,116.0,0.0,4834.0,4.0,4838.0
6,117.0,0.0,98.0,13.0,111.0
7,118.0,0.0,0.0,173.0,173.0
8,916.0,20941.0,63410.0,271.0,84622.0
